**Métodos iniciales**

Ejecuta estas celdas antes de comenzar. Incluyen funciones clave y configuración necesaria para que el resto del notebook funcione correctamente.

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
     
if not os.getenv("OPENAI_API_KEY"):
    print("Error: La variable de entorno OPENAI_API_KEY no está definida.")

print("OPENAI_API_KEY cargada correctamente.")

OPENAI_API_KEY cargada correctamente.


In [7]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [8]:
from minsearch import AppendableIndex

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [14]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [12]:
def add_entry(question, answer):
    doc = {
        'question': question,
        'text': answer,
        'section': 'user added',
        'course': 'data-engineering-zoomcamp'
    }
    index.append(doc)

# PydanticAI

**¿Qué es PydanticAI y cómo se relaciona con los agentes de IA?**

**PydanticAI** es una librería de Python creada por los mismos desarrolladores de **Pydantic** (muy popular para validar y manejar datos en Python).  
Su objetivo es facilitar la construcción de **agentes de inteligencia artificial** que interactúan con modelos de lenguaje (LLMs) de forma **estructurada, validada y confiable**.

En el contexto de IA, un **agente** es un sistema que puede:
- Recibir una **instrucción o pregunta**.
- Decidir **qué acciones tomar**.
- Usar **herramientas** o funciones externas.
- Generar una **respuesta final** al usuario.

PydanticAI ayuda a que la **comunicación entre el agente y el LLM** sea más clara y segura:
- **Estructurando datos**: Se definen modelos con tipos de datos (`str`, `int`, `list`, etc.) para que el LLM devuelva información siguiendo ese formato.
- **Validando resultados**: Si el LLM envía algo que no cumple con el formato, la librería lo detecta y puede pedirle que lo corrija.
- **Reduciendo errores**: Al tener datos validados, es más fácil integrar el LLM con otros sistemas o bases de datos.

**Relación con los Agentes de IA**
PydanticAI actúa como un **puente** entre:
1. El **modelo de lenguaje** (que genera texto o datos).
2. El **código del agente** (que decide qué hacer con esa información).

Esto permite que los agentes:
- Sepan exactamente qué información esperar del LLM.
- Manejen mejor los flujos de trabajo.
- Sean más **predecibles y robustos** en entornos reales.

PydanticAI es como un “traductor” y “verificador” que ayuda a los agentes de IA a comunicarse con los LLMs de forma más confiable, evitando datos malformados y haciendo que el desarrollo sea más ordenado.

Importa dos componentes clave de la librería `pydantic_ai`:

**Agent**

Es la clase que representa a un **agente de inteligencia artificial**.  
Se encarga de:
- Conectarse con un modelo de lenguaje (LLM).
- Recibir instrucciones o preguntas.
- Decidir qué acciones ejecutar.
- Retornar una respuesta validada y estructurada.  

Es, en esencia, el **cerebro** que coordina el comportamiento del asistente de IA.

**RunContext**

Es un objeto que almacena el **contexto de ejecución** mientras el agente procesa una solicitud.  
Incluye información como:
- La entrada actual del usuario.
- El historial de interacciones.
- Resultados obtenidos en pasos previos.  

Funciona como la **memoria temporal** del agente, necesaria para coordinar tareas que requieren varios pasos o llamadas a funciones.

**En resumen:**  
`Agent` dirige la lógica y las decisiones del asistente, mientras que `RunContext` conserva y gestiona la información que el agente necesita durante la ejecución.


In [1]:
from pydantic_ai import Agent, RunContext

**1. `developer_prompt`**
- Es un texto que define el **rol e instrucciones iniciales** para el agente.
- En este caso, el agente debe comportarse como un **asistente de enseñanza** para un curso.
- Este mensaje se usa como **`system_prompt`**, que orienta las respuestas del modelo desde el inicio.

**2. Creación del `Agent`**
- Se instancia un agente con:
  - **Modelo:** `'openai:gpt-4o-mini'` (un modelo de lenguaje de OpenAI optimizado para rapidez y bajo costo).
  - **`system_prompt`:** el contenido de `developer_prompt`, que fija el contexto y tono que debe seguir el agente.
- Este agente, una vez ejecutado, recibirá preguntas de estudiantes y las responderá según las instrucciones del `system_prompt`.

In [9]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.
When using FAQ, perform deep topic exploration: make one request to FAQ,
and then based on the results, make more requests.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

In [29]:
chat_agent = Agent(  
    'openai:gpt-4o-mini',
    system_prompt=developer_prompt
)

Define **dos herramientas** (`tools`) que el agente (`chat_agent`) puede usar para interactuar con una base de Preguntas Frecuentes (FAQ).  
En `pydantic_ai`, un *tool* es una función que el agente puede invocar cuando lo considere necesario durante una conversación.

**1. `search_tool`**
- **Función:** Permite al agente **buscar** entradas relevantes en la FAQ basándose en una consulta del usuario.
- **Parámetros:**
  - `ctx` (`RunContext`): El contexto actual de ejecución, que guarda datos de la interacción en curso.
  - `query` (`str`): Texto que el usuario quiere buscar.
- **Retorno:** Un diccionario con los resultados de búsqueda (máximo 5), incluyendo información de relevancia e identificadores de las respuestas.
- **Comportamiento:**  
  1. Muestra en consola la consulta realizada.  
  2. Llama a la función `search(query)` para obtener los resultados.  

**2. `add_entry_tool`**
- **Función:** Permite al agente **añadir** una nueva pregunta y respuesta a la FAQ.
- **Parámetros:**
  - `ctx` (`RunContext`): Contexto de ejecución actual.
  - `question` (`str`): Texto de la nueva pregunta.
  - `answer` (`str`): Respuesta o explicación correspondiente.
- **Retorno:** No devuelve datos (`None`), pero actualiza la FAQ.
- **Comportamiento:**  
  Llama a la función `add_entry(question, answer)` para registrar la nueva entrada, marcándola como contenido añadido por el usuario.

**Resumiendo:**
- `search_tool` → Busca información en la FAQ.  
- `add_entry_tool` → Agrega nuevas preguntas y respuestas a la FAQ.  
Ambas funciones están registradas como herramientas que el agente puede usar de forma autónoma cuando detecte que son necesarias para responder mejor al usuario.


In [11]:
from typing import Dict


@chat_agent.tool
def search_tool(ctx: RunContext, query: str) -> Dict[str, str]:
    """
    Search the FAQ for relevant entries matching the query.

    Parameters
    ----------
    query : str
        The search query string provided by the user.

    Returns
    -------
    list
        A list of search results (up to 5), each containing relevance information 
        and associated output IDs.
    """
    print(f"search('{query}')")
    return search(query)

In [30]:
@chat_agent.tool
def add_entry_tool(ctx: RunContext, question: str, answer: str) -> None:
    """
    Add a new question-answer entry to FAQ.

    This function creates a document with the given question and answer, 
    tagging it as user-added content.

    Parameters
    ----------
    question : str
        The question text to be added to the index.

    answer : str
        The answer or explanation corresponding to the question.

    Returns
    -------
    None
    """
    return add_entry(question, answer)

In [15]:
user_prompt = "I just discovered the course. Can I join now?"
agent_run = await chat_agent.run(user_prompt)
print(agent_run.output)

search('enrollment deadlines')
Yes, you can still join the course even if it has already started. According to the information I've found, you're eligible to participate and submit homework without registering. However, please note that there will be deadlines for turning in the final projects, so it's best not to leave everything until the last minute.

For the most accurate details regarding deadlines, you can check the latest information [here](https://docs.google.com/spreadsheets/d/e/2PACX-1vQACMLuutV5rvXg5qICuJGL-yZqIV0FBD84CxPdC5eZHf8TfzB-CJT_3Mo7U7oGVTXmSihPgQxuuoku/pubhtml).

Do you have any specific questions about the course content or requirements?


In [17]:
user_prompt = "How do I do my best for module 1?"
agent_run = await chat_agent.run(user_prompt)
print(agent_run.output)

search('how to excel in module 1')
search('how to succeed in module 1 data-engineering-zoomcamp')
To excel in Module 1 of the Data Engineering Zoomcamp, here are some tips and insights gathered from the FAQ entries:

1. **Understand Your Tools:** Ensure that you have a solid understanding of Docker and Terraform, as this module focuses on these essential tools. Familiarizing yourself with Docker commands and how to write Dockerfiles will be crucial.

2. **Docker Setup:** If you encounter build errors when working with Docker (like permission issues), make sure to check your directory permissions. For example, running `sudo chown -R $USER dir_path` can solve context errors during builds. 

3. **Python Modules:** It's essential to ensure that all required Python modules are installed. If you receive a `ModuleNotFoundError`, for instance, with the `psycopg2` module, you can resolve it by running:
   ```
   pip install psycopg2-binary
   ```
   If you still face issues post-installation, c